# Post-processing (classic)


In [55]:
import csv
import hashlib
import io
import json
import os
import re
import signal
import subprocess
import sys
import warnings
from datetime import datetime
from itertools import zip_longest
from pathlib import Path, PurePath
from pprint import pprint

import numpy as np
import pandas as pd
import requests
import requests_cache
import seaborn as sns

# from wand.image import Image
from IPython.display import Image
from nltk.metrics import edit_distance

In [56]:
print(requests_cache.__file__)

/nix/store/cjgzln8xia3l88krlry3mjrvz6nx1a6k-python3-3.8.8-env/lib/python3.8/site-packages/requests_cache/__init__.py


In [57]:
requests_cache.install_cache("pfocr_cache")

In [58]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [59]:
sns.set_style("dark")

In [60]:
import Levenshtein
import Levenshtein_search
import polyleven

## Load Data

In [61]:
data_dir = Path("../data")

### Load OCR 2020 Results

In [62]:
%sql postgresql:///pfocr20200224

In [63]:
%%sql figures2020 << SELECT id, filepath, figure_number, caption, resolution, hash
FROM figures;

 * postgresql:///pfocr20200224
114176 rows affected.
Returning data to local variable figures2020


In [64]:
figures2020_df = figures2020.DataFrame()

In [65]:
figures2020_df

,id,filepath,figure_number,caption,resolution,hash
0,99980,/home/ariutta/Documents/pfocr/data/images_ocre...,nihms631297f1,None,150,f1251c61cdae166d9efe021a38a26df030e53cc0c8b446...
1,99981,/home/ariutta/Documents/pfocr/data/images_ocre...,zam0120900500001,None,72,99bfb01f132886e392fbe9987723edf477d1480eb116a1...
2,99982,/home/ariutta/Documents/pfocr/data/images_ocre...,nutrients-09-01243-g007,None,600,a34de0ab2744b046c9a2d927676b234d308327c615bcc4...
3,99983,/home/ariutta/Documents/pfocr/data/images_ocre...,EMS83014-f003,None,96,7f4fafb790d9b92b98e3573f7d1edef1b9990e67d3af6d...
4,99984,/home/ariutta/Documents/pfocr/data/images_ocre...,12014_2017_9165_Fig1_HTML,None,72,cae8e2ac3e341cec7e41c21cbdb2317999a1bfbb0bc7bc...
...,...,...,...,...,...,...
114171,17463,/home/ariutta/Documents/pfocr/data/pmc/2019110...,kpsb-11-01-1119962-g016,None,200,d40168e350337ab4e9f18a8eacbcb1b233eeaa703e11b9...
114172,17464,/home/ariutta/Documents/pfocr/data/pmc/2019110...,1471-2407-14-217-3,None,600,1ec4ad03dd5295e97fe7f1718cf609cb21d263341c12ec...
114173,17466,/home/ariutta/Documents/pfocr/data/pmc/2019110...,gr3,None,113,581cebd1c383b70ba61359690b58ca69b1e3f978dd447f...
114174,17467,/home/ariutta/Documents/pfocr/data/pmc/2019110...,nihms491534f1,None,150,86a3b076b363fdf036689937015d97eeae8d1e664fa1da...


In [66]:
figures2020_df.head(1)["filepath"].to_list()

['/home/ariutta/Documents/pfocr/data/images_ocred_20200224/PMC4183209__nihms631297f1.jpg']

In [67]:
figures2020_df["pfocr_id"] = figures2020_df["filepath"].apply(
    lambda f: PurePath(f).name
)

Note we have a few instances of the same figure being stored in different directories.

In [68]:
with pd.option_context("display.max_colwidth", None):
    display(
        figures2020_df[
            figures2020_df["pfocr_id"].duplicated(keep=False)
        ].sort_values(by=["pfocr_id", "filepath"])
    )

,id,filepath,figure_number,caption,resolution,hash,pfocr_id
7830,107284,/home/ariutta/Documents/pfocr/data/images_ocred_20200224/PMC2099484__1471-2105-8-S7-S16-8.jpg,1471-2105-8-S7-S16-8,None,100,19c81f2cb6424cd17279e8bf2910235281e364e9ea89e965504be5fe6d9394f1,PMC2099484__1471-2105-8-S7-S16-8.jpg
89090,90587,/home/ariutta/Documents/pfocr/data/pmc/20191102_93k/images/PMC2099484__1471-2105-8-S7-S16-8.jpg,1471-2105-8-S7-S16-8,None,100,19c81f2cb6424cd17279e8bf2910235281e364e9ea89e965504be5fe6d9394f1,PMC2099484__1471-2105-8-S7-S16-8.jpg
2525,102341,/home/ariutta/Documents/pfocr/data/images_ocred_20200224/PMC2225983__1752-0509-1-56-4.jpg,1752-0509-1-56-4,None,94,b6ae057df17f7e9a50e963052785274bb294601a3edfcff11408d6c4f3df578f,PMC2225983__1752-0509-1-56-4.jpg
93084,94848,/home/ariutta/Documents/pfocr/data/pmc/20191102_93k/images/PMC2225983__1752-0509-1-56-4.jpg,1752-0509-1-56-4,None,94,b6ae057df17f7e9a50e963052785274bb294601a3edfcff11408d6c4f3df578f,PMC2225983__1752-0509-1-56-4.jpg
21952,120532,/home/ariutta/Documents/pfocr/data/images_ocred_20200224/PMC2242820__pcbi.0040028.g003.jpg,pcbi.0040028.g003,None,1282,344e1e6bfd77363b08c1b5330a427692a041451ef1fae9e99f6b1bea0f8952cf,PMC2242820__pcbi.0040028.g003.jpg
...,...,...,...,...,...,...,...
87926,89327,/home/ariutta/Documents/pfocr/data/pmc/20191102_93k/images/PMC6694157__41598_2019_48372_Fig6_HTML.jpg,41598_2019_48372_Fig6_HTML,None,96,0ee22cb93cbd7e1e238af3c8a0b5fce64bafa8ee15b21daa2a31c140bbce7f4c,PMC6694157__41598_2019_48372_Fig6_HTML.jpg
11606,110821,/home/ariutta/Documents/pfocr/data/images_ocred_20200224/PMC6727561__12885_2019_6052_Fig4_HTML.jpg,12885_2019_6052_Fig4_HTML,None,96,f5962854dad781e76aa6face4170123e908b6e0d0e0e221bedbc10cafa385256,PMC6727561__12885_2019_6052_Fig4_HTML.jpg
38323,36042,/home/ariutta/Documents/pfocr/data/pmc/20191102_93k/images/PMC6727561__12885_2019_6052_Fig4_HTML.jpg,12885_2019_6052_Fig4_HTML,None,96,f5962854dad781e76aa6face4170123e908b6e0d0e0e221bedbc10cafa385256,PMC6727561__12885_2019_6052_Fig4_HTML.jpg
4498,104176,/home/ariutta/Documents/pfocr/data/images_ocred_20200224/PMC6797170__pone.0223840.g006.jpg,pone.0223840.g006,None,300,d13a0716aade67ccf34140131c3edb5bdf07ddad3f6d4e9e63f5381e398cbf3d,PMC6797170__pone.0223840.g006.jpg


We get `114176` rows, `114117` unique pfocr_ids and `114111` unique images (by SHA256 hash):

In [69]:
figures2020_pfocr_ids = figures2020_df["pfocr_id"].to_list()
print(len(figures2020_pfocr_ids))

unique_figures2020_pfocr_ids = set(figures2020_pfocr_ids)
print(len(unique_figures2020_pfocr_ids))

figures2020_hashes = figures2020_df["hash"].to_list()
unique_figures2020_hashes = set(figures2020_hashes)
print(len(unique_figures2020_hashes))

114176
114117
114111


### Load OCR 2021 Results

and get the subset of pathway figures not in the 2020 set.

#### Load the RDS

In [71]:
import rpy2.robjects as ro
from rpy2.robjects import default_converter, pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.lib.dplyr import DataFrame
from rpy2.robjects.packages import importr

from functools import partial

from rpy2.ipython import html

html.html_rdataframe = partial(html.html_rdataframe, table_class="docutils")

In [72]:
pandas2ri.activate()
base = importr("base")
readRDS = ro.r["readRDS"]
saveRDS = ro.r["saveRDS"]

/nix/store/cjgzln8xia3l88krlry3mjrvz6nx1a6k-python3-3.8.8-env/lib/python3.8/site-packages/rpy2/robjects/vectors.py:980: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '


In [73]:
def rds2pandas(rds_path):
    r_df = readRDS(str(rds_path))
    with localconverter(ro.default_converter + pandas2ri.converter):
        pandas_df = ro.conversion.rpy2py(r_df)
    return pandas_df

In [74]:
def pandas2rds(pandas_df, rds_path):
    with localconverter(default_converter + pandas2ri.converter) as cv:
        r_df = DataFrame(pandas_df)

    saveRDS(r_df, str(rds_path))

In [75]:
target_date = "20210513"
pmc_df = (
    rds2pandas(
        data_dir.joinpath(
            f"imagesdocsum_pathway_queries/{target_date}/pmc.df.all.rds"
        )
    )
    .reset_index()
    .rename(
        columns={
            "index": "pmc_search_index",
            "figid": "pfocr_id",
            "pmcid": "pmc_id",
            "filename": "figure_filename",
            "number": "figure_number",
            "figtitle": "figure_title",
            "papertitle": "paper_title",
            "caption": "figure_caption",
            "figlink": "relative_figure_page_url",
            "reftext": "reference",
            # the RDS doesn't have a year column
            # "year": "publication_year",
        }
    )
)

pmc_df["paper_url"] = (
    "https://www.ncbi.nlm.nih.gov/pmc/articles/" + pmc_df["pmc_id"]
)

pmc_df["figure_page_url"] = (
    "https://www.ncbi.nlm.nih.gov" + pmc_df["relative_figure_page_url"]
)

pmc_df["figure_thumbnail_url"] = (
    "https://www.ncbi.nlm.nih.gov/pmc/articles/"
    + pmc_df["pmc_id"]
    + "/bin/"
    + pmc_df["figure_filename"]
)

pmc_df["pfocr_year"] = 2021

pmc_df.drop(
    columns=[
        "relative_figure_page_url",
        "figure_filename",
    ],
    inplace=True,
)

pmc_df

,pmc_search_index,pfocr_id,pmc_id,figure_number,figure_title,paper_title,figure_caption,reference,paper_url,figure_page_url,figure_thumbnail_url,pfocr_year
0,1,PMC7970325__gr12.jpg,PMC7970325,Figure 12,Average PAL values for 36 differential DNA rep...,DNA repair pathway activation features in foll...,Average PAL values for 36 differential DNA rep...,"Uliana Vladimirova, et al. Heliyon. 2021 Mar;7...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,2021
1,2,PMC7847494__ijcep0014-0097-f3.jpg,PMC7847494,Figure 3,GSEA enrichment analysis of PDK2-4 in breast c...,Expression and clinical significance of PDK fa...,GSEA enrichment analysis of PDK2-4 in breast c...,"Jian Xu, et al. Int J Clin Exp Pathol. 2021;14...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,2021
2,3,PMC6137129__fimmu-09-02055-g0006.jpg,PMC6137129,Figure 6,Overrepresented KEGG metabolic and signaling p...,Molecular and Cellular Response to Experimenta...,Overrepresented KEGG metabolic and signaling p...,"Ivana Bušelić, et al. Front Immunol. 2018;9:2055.",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,2021
3,4,PMC6721529__cells-08-00806-g003.jpg,PMC6721529,Figure 3,Canonical pathway analysis of proteins identif...,Impact of Heat Shock Protein 90 Inhibition on ...,Canonical pathway analysis of proteins identif...,"Ángela Marrugal, et al. Cells. 2019 Aug;8(8):806.",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,2021
4,5,PMC7970325__gr17.jpg,PMC7970325,Figure 17,PAL distribution for 38 DNA repair pathways in...,DNA repair pathway activation features in foll...,PAL distribution for 38 DNA repair pathways in...,"Uliana Vladimirova, et al. Heliyon. 2021 Mar;7...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
124459,124560,PMC7352181__cancers-12-01457-g002.jpg,PMC7352181,Figure 2,Angiotensin-associated pathways associated wit...,Renin-Angiotensin System in Lung Tumor and Mic...,Angiotensin-associated pathways associated wit...,"Maria Joana Catarata, et al. Cancers (Basel). ...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,2021
124460,124561,PMC6952829__bloodBLD2019001438f2.jpg,PMC6952829,Figure 2,Concurrent activation of NF-κB and Notch signa...,Coactivation of NF-κB and Notch signaling is s...,Concurrent activation of NF-κB and Notch signa...,"Yan Xiu, et al. Blood. 2020 Jan 9;135(2):108-120.",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,2021
124461,124562,PMC7777525__gr2.jpg,PMC7777525,Fig. 2,Schematic representation of type I to type VI ...,Secrete or perish: The role of secretion syste...,Schematic representation of type I to type VI ...,"Cristina E. Alvarez-Martinez, et al. Comput St...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,2021
124462,124563,PMC6092289__TJP-596-3469-g003.jpg,PMC6092289,Figure 2,,The negotiated equilibrium model of spinal cor...,"A, left: soleus EMG is monitored 24 h per day ...",Jonathan R. Wolpaw. J Physiol. 2018 Aug 15;596...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,2021


#### Load from file

In [76]:
target_date = "20210513"
images_dir = Path(f"../data/images/{target_date}")

figure_paths = list()
for ext in ("*.jpg", "*.jpeg", "*.png"):
    for f in images_dir.rglob(ext):
        figure_paths.append(f)
total_figure_path_count = len(figure_paths)
print(f"total_figure_path_count: {total_figure_path_count}")

directory_data_path = Path(
    f"../data/images/{target_date}/collected_directory_data.json"
)

if directory_data_path.exists():
    with directory_data_path.open("r") as f:
        figures2021_data = json.load(f)
else:
    figures2021_data = list()

    for figure_path in figure_paths:
        automl_output_path = figure_path.with_name(
            f"{figure_path.stem}_automl.json"
        )
        ocr_output_path = figure_path.with_name(f"{figure_path.stem}_ocr.json")

        row = {
            "pfocr_id": figure_path.name,
            "figure_path": str(figure_path),
            "ocr_output_path": str(ocr_output_path),
            "automl_output_path": str(automl_output_path),
        }

        if automl_output_path.exists():
            row["automl_succeeded"] = True
            with automl_output_path.open("r") as f:
                automl_output = json.load(f)
                row["classification"] = automl_output["displayName"]
                row["pathway_score"] = automl_output["classification"]["score"]

        if ocr_output_path.exists():
            row["ocr_succeeded"] = True

        m = hashlib.sha256()
        with open(figure_path, "rb") as f:
            m.update(f.read())
        figure_hash = m.hexdigest()

        row["hash"] = figure_hash
        row["new"] = figure_hash not in figures2020_hashes

        figures2021_data.append(row)

    with directory_data_path.open("w", encoding="utf8") as f:
        json.dump(figures2021_data, f, ensure_ascii=False)

print(len(figures2021_data))

total_figure_path_count: 124447
124447


In [77]:
on_disk_df = pd.DataFrame(figures2021_data)
on_disk_df

,pfocr_id,figure_path,ocr_output_path,automl_output_path,automl_succeeded,classification,classification_score,ocr_succeeded,hash,new
0,PMC5814228__oncotarget-09-6499-g006.jpg,../data/images/20210513/PMC5814228__oncotarget...,../data/images/20210513/PMC5814228__oncotarget...,../data/images/20210513/PMC5814228__oncotarget...,True,pathway,0.875032,True,1c9a7731d9d4a5b956d84bbc0070661cf7a0c66b8964db...,False
1,PMC6449403__41598_2019_42220_Fig7_HTML.jpg,../data/images/20210513/PMC6449403__41598_2019...,../data/images/20210513/PMC6449403__41598_2019...,../data/images/20210513/PMC6449403__41598_2019...,True,other,0.939483,NaN,e4511fb4f5eacc744c9faf678bd49b0362069aafc98002...,True
2,PMC6161541__fphar-09-01071-g006.jpg,../data/images/20210513/PMC6161541__fphar-09-0...,../data/images/20210513/PMC6161541__fphar-09-0...,../data/images/20210513/PMC6161541__fphar-09-0...,True,other,0.936534,NaN,b09c82c0a8fb6b3306c2a6d8599d54ca41c935e38e2c68...,True
3,PMC6629942__bsr-39-bsr20191210-g2.jpg,../data/images/20210513/PMC6629942__bsr-39-bsr...,../data/images/20210513/PMC6629942__bsr-39-bsr...,../data/images/20210513/PMC6629942__bsr-39-bsr...,True,other,0.968381,NaN,31f55f367a60d38f2f09582ba5a4e54d75793d98905b52...,False
4,PMC7520991__MBO3-9-e1098-g005.jpg,../data/images/20210513/PMC7520991__MBO3-9-e10...,../data/images/20210513/PMC7520991__MBO3-9-e10...,../data/images/20210513/PMC7520991__MBO3-9-e10...,True,other,0.631791,NaN,edcd25e5999e5542120e391db0c6c87c89308cc763789b...,True
...,...,...,...,...,...,...,...,...,...,...
124442,PMC5880682__nihms931867f3.jpg,../data/images/20210513/PMC5880682__nihms93186...,../data/images/20210513/PMC5880682__nihms93186...,../data/images/20210513/PMC5880682__nihms93186...,True,other,0.999391,NaN,5b5a3ec17542c5dc1758d56747aa73b9f14b078da157d4...,True
124443,PMC6526157__WJG-25-2188-g005.jpg,../data/images/20210513/PMC6526157__WJG-25-218...,../data/images/20210513/PMC6526157__WJG-25-218...,../data/images/20210513/PMC6526157__WJG-25-218...,True,pathway,0.644881,True,0cca3c2cff1f9edc584c034f821f6561aeae9a7ccfaebf...,False
124444,PMC8043451__nihpp-2020.08.06.240325-f0001.jpg,../data/images/20210513/PMC8043451__nihpp-2020...,../data/images/20210513/PMC8043451__nihpp-2020...,../data/images/20210513/PMC8043451__nihpp-2020...,True,other,0.942510,NaN,37998a952826603432d96df08a6971a2cff8f2406dfc0e...,True
124445,PMC7859918__OR-45-03-0921-g00.jpg,../data/images/20210513/PMC7859918__OR-45-03-0...,../data/images/20210513/PMC7859918__OR-45-03-0...,../data/images/20210513/PMC7859918__OR-45-03-0...,True,other,0.962785,NaN,a355628d0163d24236e877edc0a4c5c3419a509854ef00...,True


In [78]:
raw_pfocr_figures_df = pmc_df.join(
    on_disk_df.set_index("pfocr_id"),
    on="pfocr_id",
    how="outer",
).rename(columns={"classification_score": "pathway_score"})
raw_pfocr_figures_df

,pmc_search_index,pfocr_id,pmc_id,figure_number,figure_title,paper_title,figure_caption,reference,paper_url,figure_page_url,...,pfocr_year,figure_path,ocr_output_path,automl_output_path,automl_succeeded,classification,pathway_score,ocr_succeeded,hash,new
0,1,PMC7970325__gr12.jpg,PMC7970325,Figure 12,Average PAL values for 36 differential DNA rep...,DNA repair pathway activation features in foll...,Average PAL values for 36 differential DNA rep...,"Uliana Vladimirova, et al. Heliyon. 2021 Mar;7...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,...,2021,../data/images/20210513/PMC7970325__gr12.jpg,../data/images/20210513/PMC7970325__gr12_ocr.json,../data/images/20210513/PMC7970325__gr12_autom...,True,other,0.929271,NaN,95341927cfd958f048c3e797dedd8355f81f5f729a0c39...,True
1,2,PMC7847494__ijcep0014-0097-f3.jpg,PMC7847494,Figure 3,GSEA enrichment analysis of PDK2-4 in breast c...,Expression and clinical significance of PDK fa...,GSEA enrichment analysis of PDK2-4 in breast c...,"Jian Xu, et al. Int J Clin Exp Pathol. 2021;14...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,...,2021,../data/images/20210513/PMC7847494__ijcep0014-...,../data/images/20210513/PMC7847494__ijcep0014-...,../data/images/20210513/PMC7847494__ijcep0014-...,True,other,0.878512,NaN,416ffa69c28b493b52170c72421908e85a87c075eeb6fd...,True
2,3,PMC6137129__fimmu-09-02055-g0006.jpg,PMC6137129,Figure 6,Overrepresented KEGG metabolic and signaling p...,Molecular and Cellular Response to Experimenta...,Overrepresented KEGG metabolic and signaling p...,"Ivana Bušelić, et al. Front Immunol. 2018;9:2055.",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,...,2021,../data/images/20210513/PMC6137129__fimmu-09-0...,../data/images/20210513/PMC6137129__fimmu-09-0...,../data/images/20210513/PMC6137129__fimmu-09-0...,True,other,0.890991,NaN,ae2f5df8fb47342ff43b643f07c2234f61f138ba110a1e...,False
3,4,PMC6721529__cells-08-00806-g003.jpg,PMC6721529,Figure 3,Canonical pathway analysis of proteins identif...,Impact of Heat Shock Protein 90 Inhibition on ...,Canonical pathway analysis of proteins identif...,"Ángela Marrugal, et al. Cells. 2019 Aug;8(8):806.",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,...,2021,../data/images/20210513/PMC6721529__cells-08-0...,../data/images/20210513/PMC6721529__cells-08-0...,../data/images/20210513/PMC6721529__cells-08-0...,True,other,0.967595,NaN,f6a17e42e20fb384fff9e0288b3f26c4a16ef52004d666...,False
4,5,PMC7970325__gr17.jpg,PMC7970325,Figure 17,PAL distribution for 38 DNA repair pathways in...,DNA repair pathway activation features in foll...,PAL distribution for 38 DNA repair pathways in...,"Uliana Vladimirova, et al. Heliyon. 2021 Mar;7...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,...,2021,../data/images/20210513/PMC7970325__gr17.jpg,../data/images/20210513/PMC7970325__gr17_ocr.json,../data/images/20210513/PMC7970325__gr17_autom...,True,other,0.946393,NaN,1c9fa4be76f84b7587bec4bb2808efde41742b23d214f8...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124459,124560,PMC7352181__cancers-12-01457-g002.jpg,PMC7352181,Figure 2,Angiotensin-associated pathways associated wit...,Renin-Angiotensin System in Lung Tumor and Mic...,Angiotensin-associated pathways associated wit...,"Maria Joana Catarata, et al. Cancers (Basel). ...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,...,2021,../data/images/20210513/PMC7352181__cancers-12...,../data/images/20210513/PMC7352181__cancers-12...,../data/images/20210513/PMC7352181__cancers-12...,True,pathway,0.956203,True,99861893d87c584816f32b88750bf8cf62f3d30cb04b6d...,True
124460,124561,PMC6952829__bloodBLD2019001438f2.jpg,PMC6952829,Figure 2,Concurre

In [79]:
raw_pfocr_figures_df.columns

Index(['pmc_search_index', 'pfocr_id', 'pmc_id', 'figure_number',
       'figure_title', 'paper_title', 'figure_caption', 'reference',
       'paper_url', 'figure_page_url', 'figure_thumbnail_url', 'pfocr_year',
       'figure_path', 'ocr_output_path', 'automl_output_path',
       'automl_succeeded', 'classification', 'pathway_score', 'ocr_succeeded',
       'hash', 'new'],
      dtype='object')

In [80]:
images_dir.joinpath("hello.txt")

PosixPath('../data/images/20210513/hello.txt')

In [81]:
exported_pfocr_figures_df = raw_pfocr_figures_df[
    raw_pfocr_figures_df["ocr_succeeded"] & raw_pfocr_figures_df["new"]
][
    [
        "pfocr_id",
        "figure_page_url",
        "figure_thumbnail_url",
        "figure_number",
        "figure_title",
        "figure_caption",
        "pmc_id",
        "paper_url",
        "paper_title",
        "reference",
        # the RDS doesn't have a year column
        # "publication_year",
        "pmc_search_index",
        "pathway_score",
        "pfocr_year",
    ]
]

pandas2rds(
    exported_pfocr_figures_df, images_dir.joinpath("pfocr_figures_2021.rds")
)

exported_pfocr_figures_df

,pfocr_id,figure_page_url,figure_thumbnail_url,figure_number,figure_title,figure_caption,pmc_id,paper_url,paper_title,reference,pmc_search_index,pathway_score,pfocr_year
11,PMC7226520__cells-09-01043-g007.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Figure 7,Comparative downstream pathway analysis of the...,Comparative downstream pathway analysis of the...,PMC7226520,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,"A STAT3 of Addiction: Adipose Tissue, Adipocyt...","Rose Kadye, et al. Cells. 2020 Apr;9(4):1043.",12,0.811027,2021
16,PMC7346062__aging-12-103262-g005.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Figure 5,KEGG pathways related to resveratrol-targeted ...,KEGG pathways related to resveratrol-targeted ...,PMC7346062,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Resveratrol promotes osteogenesis and alleviat...,"Tao Yu, et al. Aging (Albany NY). 2020 Jun 15;...",17,0.943144,2021
21,PMC7063815__13578_2020_396_Fig2_HTML.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Fig. 2,Interaction between tumor metabolism and the m...,Interaction between tumor metabolism and the m...,PMC7063815,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,mTOR signaling pathway and mTOR inhibitors in ...,"Zhilin Zou, et al. Cell Biosci. 2020;10:31.",22,0.943767,2021
22,PMC6497965__zbc0191904900006.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,Figure 6,A proposed signaling cascade for phosphorylate...,A proposed signaling cascade for phosphorylate...,PMC6497965,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,Phosphorylation of proliferating cell nuclear ...,"Bo Peng, et al. J Biol Chem. 2019 Apr 26;294(1...",23,0.603429,2021
23,PMC7538683__bsr-40-bsr20202711-g5.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Figure 5,,(A) Cell cycle signaling pathway is significan...,PMC7538683,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Bioinformatics analysis and experimental valid...,"Jiajia Chen, et al. Biosci Rep. 2020 Oct 30;40...",24,0.863582,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124414,PMC7803631__gr3a.jpg,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,,Fig,Cell free biosynthesis for erythromycin A. The...,PMC7803631,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Complex natural product production methods and...,"Dongwon Park, et al. Synth Syst Biotechnol. 20...",124515,0.648432,2021
124421,PMC7359798__gr2_lrg.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Figure 2,RNA Sensing and ResponseNon-comprehensive over...,RNA Sensing and ResponseNon-comprehensive over...,PMC7359798,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Immune Sensing Mechanisms that Discriminate Se...,"Eva Bartok, et al. Immunity. 2020 Jul 14;53(1)...",124522,0.922250,2021
124439,PMC7530268__fonc-10-586530-g0001.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Figure 1,The ferroptotic cascade,The ferroptotic cascade. Accumulation of free ...,PMC7530268,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,From Iron Chelation to Overload as a Therapeut...,"Eric Grignano, et al. Front Oncol. 2020;10:586...",124540,0.952995,2021
124445,PMC7466447__fcell-08-00766-g001.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,FIGURE 1,Immunoregulatory functions of TEC in the TME,Immunoregulatory functions of TEC in the TME. ...,PMC7466447,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Tumor Endothelial Cells (TECs) as Potential Im...,"Laurenz Nagl, et al. Front Cell Dev Biol. 2020...",124546,0.933006,2021


#### Compare the 2021 RDS with what's on file

- `124464` figures from PMC search (no duplicate pfocr_ids)
- `124447` downloaded (some 404'd) (no duplicate pfocr_ids)
- `124441` AutoML'd (some reject as invalid JPGs)
- `28701` classified as pathway and OCR'd

In [82]:
print(len([f for f in images_dir.rglob("*.jpg")]))
print(len(set([f for f in images_dir.rglob("*.jpg")])))
print(len([f for f in images_dir.rglob("*_automl.json")]))
print(len([f for f in images_dir.rglob("*_ocr.json")]))

124447
124447
124441
28700


In [83]:
rds_pfocr_ids = pmc_df["pfocr_id"].to_list()
print(len(rds_pfocr_ids))
unique_rds_pfocr_ids = set(pmc_df["pfocr_id"].to_list())
print(len(unique_rds_pfocr_ids))

pfocr_ids = [x.name for x in figure_paths]
print(len(pfocr_ids))
unique_pfocr_ids = set(pfocr_ids)
print(len(unique_pfocr_ids))

124464
124464
124447
124447


### Invalid Images

Some of the JPGs either didn't fully download or didn't download at all.

In [84]:
for x in figures2021_data:
    figure_path = x["figure_path"]
    if not Path(x["automl_output_path"]).exists():
        print(figure_path)
        display(Image(figure_path))

../data/images/20210513/PMC7404177__ijms-21-05147-g002.jpg


../data/images/20210513/PMC8009799__467_2020_4588_Fig2_HTML.jpg


../data/images/20210513/PMC7753985__JCMM-24-13949-g002.jpg


../data/images/20210513/PMC7273476__BMRI2020-7532306.005.jpg


../data/images/20210513/PMC6435557__bsr-38-bsr20180598-g5.jpg


../data/images/20210513/PMC7932688__elife-65552-fig1.jpg


### Get the "new" figures


There are `15955` "new" figures (those in the 2021 batch but not the 2020 batch). None of the "new" figures are duplicates of each other.

In [85]:
figures2021_new_pfocr_ids = [
    x["pfocr_id"]
    for x in figures2021_data
    if x["new"]
    and Path(x["ocr_output_path"]).exists()
    and x["classification"] == "pathway"
]
print(len(figures2021_new_pfocr_ids))
unique_figures2021_new_pfocr_ids = set(figures2021_new_pfocr_ids)
print(len(unique_figures2021_new_pfocr_ids))

15955
15955


## Extract Data


### Genes

In [86]:
with open(data_dir.joinpath("lexicon2020.json"), "r") as f:
    lexicon2020_df = pd.read_json(f)

In [87]:
lexicon2020_df

,symbol,ncbigene_id,source
0,S863-7,2,hgnc_alias_symbol
1,FWP007,2,hgnc_alias_symbol
2,CPAMD5,2,hgnc_alias_symbol
3,A2M,2,hgnc_symbol
4,AAC2,10,hgnc_prev_symbol
...,...,...,...
60239,C17orf113,110806298,hgnc_symbol
60240,CSNKA2IP,111064647,hgnc_symbol
60241,C19orf85,111064650,hgnc_symbol
60242,IFITMD9,111216276,hgnc_alias_symbol


In [88]:
ncbigene_ids_by_symbol = (
    lexicon2020_df[["ncbigene_id", "symbol"]]
    .set_index("symbol")
    .to_dict()["ncbigene_id"]
)

In [89]:
symbol_characters = set()
for symbol in ncbigene_ids_by_symbol.keys():
    for c in symbol:
        symbol_characters.add(c)
print(len(symbol_characters))

74


In [90]:
figures2021_new_data = [
    x
    for x in figures2021_data
    if x["new"]
    and Path(x["ocr_output_path"]).exists()
    and x["classification"] == "pathway"
]
print(len(figures2021_new_data))

15955


Here's how we did it in 2020:
```
./pfocr.py match -n stop -n nfkc -n deburr -m expand -m root -n swaps -n alphanumeric
```

In [91]:
# acceptable_characters = symbol_characters.union({"-", "/", "&", ",", " "})

In [92]:
run_timestamp = datetime.now().strftime("%Y%m%d%H%M")

In [93]:
import transforms

results_path = images_dir.joinpath(f"results{run_timestamp}.tsv")
successes_path = images_dir.joinpath(f"successes{run_timestamp}.txt")
fails_path = images_dir.joinpath(f"fails{run_timestamp}.txt")

next_pfocr_id_path = images_dir.joinpath("next_pfocr_id_path.txt")


with open(next_pfocr_id_path, "w") as f:
    f.write("")

with open(results_path, "w") as f:
    f.write(
        "\t".join(["pfocr_id", "ncbigene_id", "word", "transformed_word"])
        + "\n"
    )

with open(successes_path, "w") as f:
    f.write("")

with open(fails_path, "w") as f:
    f.write("")


def attempt_match(
    matches_data,
    matches,
    transforms_applied,
    figure_id,
    word,
    symbol_id,
    transformed_word,
):
    if transformed_word:
        matches.add(transformed_word)
        with open(results_path, "a") as f:
            f.write(
                "\t".join([figure_id, str(symbol_id), word, transformed_word])
                + "\n"
            )

        matches_data.append(
            {
                "figure_id": figure_id,
                "ncbigene_id": symbol_id,
                "word": word,
                "transformed_word": transformed_word,
                "transforms_applied": transforms_applied,
            }
        )


transforms_to_apply = [
    {
        "name": "stop",
        "transform": transforms.stop.stop,
    },
    {
        "name": "homoglyphs2ascii",
        "transform": lambda word: transforms.homoglyphs2ascii.homoglyphs2ascii(
            word, symbol_characters
        ),
    },
    {
        "name": "nfkc",
        "transform": transforms.nfkc.nfkc,
    },
    {
        "name": "deburr",
        "transform": transforms.deburr.deburr,
    },
    {
        "name": "expand",
        "transform": transforms.expand.expand,
    },
    {
        "name": "root",
        "transform": transforms.root.root,
    },
    {
        "name": "swaps",
        "transform": transforms.swaps.swaps,
    },
    {
        "name": "alphanumeric",
        "transform": transforms.alphanumeric.alphanumeric,
    },
]


def match(matches_data, figure_id, symbol_ids_by_symbol, ocr_text):
    with open(next_pfocr_id_path, "a") as f:
        f.write(figure_id)

    successes = list()
    fails = list()

    for line in ocr_text.split("\n"):
        matches = set()

        words = set()
        words.add(line.replace(" ", ""))

        for w in line.split(" "):
            words.add(w)

        for word in words:
            transforms_applied = []
            transformed_words = [word]
            for transform_to_apply in transforms_to_apply:
                transforms_applied.append(transform_to_apply["name"])
                for transformed_word_prev in transformed_words:
                    transformed_words = []
                    for transformed_word in transform_to_apply["transform"](
                        transformed_word_prev
                    ):
                        # perform match for original and uppercased words (see elif)

                        try:
                            if transformed_word in symbol_ids_by_symbol:
                                attempt_match(
                                    matches_data,
                                    matches,
                                    transforms_applied,
                                    figure_id,
                                    word,
                                    symbol_ids_by_symbol[transformed_word],
                                    transformed_word,
                                )
                            elif (
                                transformed_word.upper() in symbol_ids_by_symbol
                            ):
                                attempt_match(
                                    matches_data,
                                    matches,
                                    transforms_applied,
                                    figure_id,
                                    word,
                                    symbol_ids_by_symbol[
                                        transformed_word.upper()
                                    ],
                                    transformed_word.upper(),
                                )
                            else:
                                transformed_words.append(transformed_word)

                        #    except TimedOutExc as e:
                        #        print "took too long"

                        except (Exception) as e:
                            print("Unexpected Error:", e)
                            print("figure_id:", figure_id)
                            print("word:", word)
                            print(
                                "transformed_word:",
                                transformed_word,
                            )
                            print(
                                "transforms_applied:",
                                transforms_applied,
                            )
                            raise

            if len(matches) == 0:
                attempt_match(
                    matches_data,
                    matches,
                    transforms_applied,
                    figure_id,
                    word,
                    None,
                    None,
                )
        if len(matches) > 0:
            successes.append(line + " => " + " & ".join(matches))
        else:
            fails.append(line)

    with open(successes_path, "w") as f:
        for success in successes:
            f.write(success + "\n")
        f.write("\n")

    with open(fails_path, "w") as f:
        for fail in fails:
            f.write(fail + "\n")
        f.write("\n")

    # print(f"{figure_id}")
    # print(f"  successes")
    # print(f"  {successes}")
    # print(f"  fails")
    # print(f"  {fails}")
    # print("")

In [94]:
my_matches_data = list()
for x in figures2021_new_data:
    figure_path = Path(x["figure_path"])

    ocr_output_path = Path(x["ocr_output_path"])
    with ocr_output_path.open("r", encoding="utf8") as f:
        ocr_output = json.load(f)
    if len(ocr_output) == 0:
        print(f"empty ocr_output_path: {ocr_output_path}")
        continue

    ocr_text = ocr_output[0]["description"]

    figure_id = x["pfocr_id"]

    # if figure_id != "PMC7905103__fphys-11-00718-g005.jpg":
    #    continue
    # display(Image(figure_path))

    try:
        match(my_matches_data, figure_id, ncbigene_ids_by_symbol, ocr_text)
    except (Exception) as e:
        print("Unexpected Error:", e)
        print("figure_id:", figure_id)
        #        print("word:", word)
        #        print(
        #            "transformed_word:",
        #            transformed_word,
        #        )
        #        print(
        #            "transforms_applied:",
        #            transforms_applied,
        #        )
        raise

matches_df = pd.DataFrame(my_matches_data)
matches_df.to_csv(str(images_dir.joinpath("results{run_timestamp}.csv")))
matches_df

empty ocr_output_path: ../data/images/20210513/PMC7271281__CMMM2020-5390107.003_ocr.json
empty ocr_output_path: ../data/images/20210513/PMC7532093__13048_2020_722_Fig7_HTML_ocr.json
empty ocr_output_path: ../data/images/20210513/PMC7160985__12885_2020_6774_Fig3_HTML_ocr.json
empty ocr_output_path: ../data/images/20210513/PMC6739048__kccy-18-19-1646561-g010_ocr.json
empty ocr_output_path: ../data/images/20210513/PMC7425190__rmdopen-2020-001258f02_ocr.json


,figure_id,ncbigene_id,word,transformed_word,transforms_applied
0,PMC8036963__cancers-13-01583-g003.jpg,3014,H2AX,H2AX,"[stop, homoglyphs2ascii, nfkc, deburr, expand,..."
1,PMC8036963__cancers-13-01583-g003.jpg,3014,H2AX,H2AX,"[stop, homoglyphs2ascii, nfkc, deburr, expand,..."
2,PMC8036963__cancers-13-01583-g003.jpg,2547,Ku70/80,KU70,"[stop, homoglyphs2ascii, nfkc, deburr, expand,..."
3,PMC8036963__cancers-13-01583-g003.jpg,7520,Ku70/80,KU80,"[stop, homoglyphs2ascii, nfkc, deburr, expand,..."
4,PMC8036963__cancers-13-01583-g003.jpg,472,ATM,ATM,"[stop, homoglyphs2ascii, nfkc, deburr, expand,..."
...,...,...,...,...,...
214150,PMC7927090__ijms-22-02194-g002.jpg,207,AKT,AKT,"[stop, homoglyphs2ascii, nfkc, deburr, expand,..."
214151,PMC7927090__ijms-22-02194-g002.jpg,4092,Smad,SMAD,"[stop, homoglyphs2ascii, nfkc, deburr, expand,..."
214152,PMC7927090__ijms-22-02194-g002.jpg,2475,MTOR,MTOR,"[stop, homoglyphs2ascii, nfkc, deburr, expand,..."
214153,PMC7927090__ijms-22-02194-g002.jpg,3791,VEGFR2,VEGFR2,"[stop, homoglyphs2ascii, nfkc, deburr, expand,..."


Save as RDS:

In [95]:
from rpy2.robjects import default_converter
from rpy2.robjects.lib.dplyr import DataFrame

output_df = matches_df.copy(deep=True)
output_df["transforms_applied"] = output_df["transforms_applied"].str.join(",")

with localconverter(default_converter + pandas2ri.converter) as cv:
    dataf = DataFrame(output_df)

saveRDS(dataf, str(images_dir.joinpath(f"results{run_timestamp}.rds")))

<rpy2.rinterface_lib.sexp.NULLType object at 0x770a6a81d300> [RTYPES.NILSXP]

In [52]:
results2021_df = pd.read_csv(
    str(images_dir.joinpath("results2021.tsv")), sep="\t"
)
print(len(results2021_df.drop_duplicates()))
results2021_df

117161


,pfocr_id,ncbigene_id,word,transformed_word
0,PMC7905103__fphys-11-00718-g005.jpg,3929,LBP,LBP
1,PMC7905103__fphys-11-00718-g005.jpg,671,BPI,BPI
2,PMC7905103__fphys-11-00718-g005.jpg,7099,TLR4,TLR4
3,PMC7905103__fphys-11-00718-g005.jpg,929,CD14,CD14
4,PMC7905103__fphys-11-00718-g005.jpg,5971,NFKB,NFKB
...,...,...,...,...
187215,PMC7927090__ijms-22-02194-g002.jpg,207,AKT,AKT
187216,PMC7927090__ijms-22-02194-g002.jpg,4092,Smad,SMAD
187217,PMC7927090__ijms-22-02194-g002.jpg,2475,MTOR,MTOR
187218,PMC7927090__ijms-22-02194-g002.jpg,3791,VEGFR2,VEGFR2


In [48]:
results202105191209_df = pd.read_csv(
    str(images_dir.joinpath("results202105191209.tsv")), sep="\t"
)
results202105191209_df

,pfocr_id,ncbigene_id,word,transformed_word
0,PMC8036963__cancers-13-01583-g003.jpg,3014,H2AX,H2AX
1,PMC8036963__cancers-13-01583-g003.jpg,3014,H2AX,H2AX
2,PMC8036963__cancers-13-01583-g003.jpg,2547,Ku70/80,KU70
3,PMC8036963__cancers-13-01583-g003.jpg,7520,Ku70/80,KU80
4,PMC8036963__cancers-13-01583-g003.jpg,472,ATM,ATM
...,...,...,...,...
145852,PMC7927090__ijms-22-02194-g002.jpg,207,AKT,AKT
145853,PMC7927090__ijms-22-02194-g002.jpg,4092,Smad,SMAD
145854,PMC7927090__ijms-22-02194-g002.jpg,2475,MTOR,MTOR
145855,PMC7927090__ijms-22-02194-g002.jpg,3791,VEGFR2,VEGFR2


In [49]:
results202105191209_df.drop_duplicates()

,pfocr_id,ncbigene_id,word,transformed_word
0,PMC8036963__cancers-13-01583-g003.jpg,3014,H2AX,H2AX
2,PMC8036963__cancers-13-01583-g003.jpg,2547,Ku70/80,KU70
3,PMC8036963__cancers-13-01583-g003.jpg,7520,Ku70/80,KU80
4,PMC8036963__cancers-13-01583-g003.jpg,472,ATM,ATM
5,PMC8036963__cancers-13-01583-g003.jpg,84126,ATRIP,ATRIP
...,...,...,...,...
145850,PMC7927090__ijms-22-02194-g002.jpg,5594,ERK,ERK
145851,PMC7927090__ijms-22-02194-g002.jpg,4792,IKB-a,IKBA
145852,PMC7927090__ijms-22-02194-g002.jpg,207,AKT,AKT
145854,PMC7927090__ijms-22-02194-g002.jpg,2475,MTOR,MTOR


In [54]:
unique_results2021_df = results2021_df.drop_duplicates()
unique_results202105191209_df = results202105191209_df.drop_duplicates()

In [ ]:
run_timestamp = datetime.now().strftime("%Y%m%d%H%M")

In [ ]:
import transforms

results_path = images_dir.joinpath(f"results{run_timestamp}.tsv")
successes_path = images_dir.joinpath(f"successes{run_timestamp}.txt")
fails_path = images_dir.joinpath(f"fails{run_timestamp}.txt")

next_pfocr_id_path = images_dir.joinpath("next_pfocr_id_path.txt")


with open(next_pfocr_id_path, "w") as f:
    f.write("")

with open(results_path, "w") as f:
    f.write(
        "\t".join(["pfocr_id", "ncbigene_id", "ocr_text", "lexicon_entry"])
        + "\n"
    )

with open(successes_path, "w") as f:
    f.write("")

with open(fails_path, "w") as f:
    f.write("")


def attempt_match(
    matches_data,
    matches,
    transforms_applied,
    pfocr_id,
    ocr_text,
    symbol_id,
    transformed_ocr_text,
):
    if transformed_ocr_text:
        matches.add(transformed_ocr_text)
        with open(results_path, "a") as f:
            f.write(
                "\t".join(
                    [pfocr_id, str(symbol_id), ocr_text, transformed_ocr_text]
                )
                + "\n"
            )

        matches_data.append(
            {
                "pfocr_id": pfocr_id,
                "ncbigene_id": symbol_id,
                "ocr_text": ocr_text,
                "lexicon_entry": transformed_ocr_text,
                "transforms_applied": transforms_applied,
            }
        )


transforms_to_apply = [
    {
        "name": "stop",
        "transform": transforms.stop.stop,
    },
    {
        "name": "homoglyphs2ascii",
        "transform": lambda ocr_text: transforms.homoglyphs2ascii.homoglyphs2ascii(
            ocr_text, symbol_characters
        ),
    },
    {
        "name": "nfkc",
        "transform": transforms.nfkc.nfkc,
    },
    {
        "name": "deburr",
        "transform": transforms.deburr.deburr,
    },
    {
        "name": "expand",
        "transform": transforms.expand.expand,
    },
    {
        "name": "root",
        "transform": transforms.root.root,
    },
    {
        "name": "swaps",
        "transform": transforms.swaps.swaps,
    },
    {
        "name": "alphanumeric",
        "transform": transforms.alphanumeric.alphanumeric,
    },
]


def match(matches_data, pfocr_id, symbol_ids_by_symbol, all_raw_ocr_text):
    with open(next_pfocr_id_path, "a") as f:
        f.write(pfocr_id)

    successes = list()
    fails = list()

    for line in all_raw_ocr_text.split("\n"):
        matches = set()

        ocr_texts = set()
        ocr_texts.add(line.replace(" ", ""))

        for w in line.split(" "):
            ocr_texts.add(w)

        for ocr_text in ocr_texts:
            transforms_applied = []
            transformed_ocr_texts = [ocr_text]
            for transform_to_apply in transforms_to_apply:
                transforms_applied.append(transform_to_apply["name"])
                for transformed_ocr_text_prev in transformed_ocr_texts:
                    transformed_ocr_texts = []
                    for transformed_ocr_text in transform_to_apply["transform"](
                        transformed_ocr_text_prev
                    ):
                        # perform match for original and uppercased ocr_texts (see elif)

                        try:
                            if transformed_ocr_text in symbol_ids_by_symbol:
                                attempt_match(
                                    matches_data,
                                    matches,
                                    transforms_applied,
                                    pfocr_id,
                                    ocr_text,
                                    symbol_ids_by_symbol[transformed_ocr_text],
                                    transformed_ocr_text,
                                )
                            elif (
                                transformed_ocr_text.upper()
                                in symbol_ids_by_symbol
                            ):
                                attempt_match(
                                    matches_data,
                                    matches,
                                    transforms_applied,
                                    pfocr_id,
                                    ocr_text,
                                    symbol_ids_by_symbol[
                                        transformed_ocr_text.upper()
                                    ],
                                    transformed_ocr_text.upper(),
                                )
                            else:
                                transformed_ocr_texts.append(
                                    transformed_ocr_text
                                )

                        #    except TimedOutExc as e:
                        #        print "took too long"

                        except (Exception) as e:
                            print("Unexpected Error:", e)
                            print("pfocr_id:", pfocr_id)
                            print("ocr_text:", ocr_text)
                            print(
                                "transformed_ocr_text:",
                                transformed_ocr_text,
                            )
                            print(
                                "transforms_applied:",
                                transforms_applied,
                            )
                            raise

            if len(matches) == 0:
                attempt_match(
                    matches_data,
                    matches,
                    transforms_applied,
                    pfocr_id,
                    ocr_text,
                    None,
                    None,
                )
        if len(matches) > 0:
            successes.append(line + " => " + " & ".join(matches))
        else:
            fails.append(line)

    with open(successes_path, "w") as f:
        for success in successes:
            f.write(success + "\n")
        f.write("\n")

    with open(fails_path, "w") as f:
        for fail in fails:
            f.write(fail + "\n")
        f.write("\n")

    # print(f"{pfocr_id}")
    # print(f"  successes")
    # print(f"  {successes}")
    # print(f"  fails")
    # print(f"  {fails}")
    # print("")

In [ ]:
my_matches_data = list()
for x in figures2021_new_data:
    figure_path = Path(x["figure_path"])

    ocr_output_path = Path(x["ocr_output_path"])
    with ocr_output_path.open("r", encoding="utf8") as f:
        ocr_output = json.load(f)
    if len(ocr_output) == 0:
        print(f"empty ocr_output_path: {ocr_output_path}")
        continue

    ocr_text = ocr_output[0]["description"]

    pfocr_id = x["pfocr_id"]

    # if pfocr_id != "PMC7905103__fphys-11-00718-g005.jpg":
    #    continue
    # display(Image(figure_path))

    try:
        match(my_matches_data, pfocr_id, ncbigene_ids_by_symbol, ocr_text)
    except (Exception) as e:
        print("Unexpected Error:", e)
        print("pfocr_id:", pfocr_id)
        #        print("word:", word)
        #        print(
        #            "transformed_word:",
        #            transformed_word,
        #        )
        #        print(
        #            "transforms_applied:",
        #            transforms_applied,
        #        )
        raise

matches_df = pd.DataFrame(my_matches_data)
matches_df.to_csv(str(images_dir.joinpath("results{run_timestamp}.csv")))
matches_df

In [38]:
[
    "ocr_text",
    "lexicon_match",
    "lexicon_match_source",
    "hgnc_symbol",
    "ncbigene_id",
    "unique_gene_count",
]


pandas2rds(
    exported_pfocr_figures_df, images_dir.joinpath("pfocr_figures_2021.rds")
)

['ocr_text',
 'lexicon_match',
 'lexicon_match_source',
 'hgnc_symbol',
 'ncbigene_id',
 'unique_gene_count']

## Extract for the sample images

In [ ]:
target_batch = "20210506sample"
images_dir = Path(f"../data/images/{target_batch}")

figure_paths = list()
for ext in ("*.jpg", "*.jpeg", "*.png"):
    for f in images_dir.rglob(ext):
        figure_paths.append(f)

results = dict()

# "el" is modern Greek
# "grc" is ancient greek
# "got" is Gothic, which maps to "Latin script model".
# .     I think that means it just tries to recognize Latin script characters.
# "fy" is western Frisian, mapping to generic Latin script model
language_hints_sets = [
    # I am calling the following "unspecified":
    [],
    #
    ["en"],
    #
    # ==============================================
    # the following lines produce identical results:
    ["el"],
    ["en", "el"],
    ["en", "grc"],
    # --------------------------
    #
    # ==============================================
    # the following lines produce identical results:
    ["got"],
    # ["got", "Grek"],
    # --------------------------
    #
    ["fy"],
    #
    # ==============================================
    # the following lines produce identical results:
    ["grc"],
    ["got", "grc"],
    ["got", "el"],
    ["got", "el", "grc"],
    # ["el", "grc"],
    # --------------------------
    #
    ["en", "got", "grc"],
    ["en", "got", "el"],
]

for figure_path in figure_paths:
    # figure_path_key = str(figure_path)
    figure_path_key = figure_path.name
    results[figure_path_key] = dict()
    for language_hints in language_hints_sets:
        text_annotations = get_text(figure_path, language_hints)

        readable_language_hints = language_hints.copy()
        if len(readable_language_hints) == 0:
            readable_language_hints.append("unspecified")

        results[figure_path_key][
            tuple(readable_language_hints)
        ] = text_annotations

        # Specifying utf8 and 'ensure_ascii=False' ensures the output file uses
        # Greek characters, not escape encoding like "\u..."
        with figure_path.with_name(
            "_".join([figure_path.stem] + readable_language_hints)
        ).with_suffix(".json").open("w", encoding="utf8") as f:
            json.dump(text_annotations, f, ensure_ascii=False)

        description = text_annotations[0]["description"]
        with figure_path.with_name(
            "_".join([figure_path.stem] + readable_language_hints)
        ).with_suffix(".txt").open("w", encoding="utf8") as f:
            f.write(repr(description))

## OLD

The following is just copied from pp_ahocorasick.ipynb:

In [2]:
hgnc_data_url = "https://www.genenames.org/cgi-bin/download/custom?col=gd_hgnc_id&col=gd_app_sym&col=gd_app_name&col=gd_status&col=gd_prev_sym&col=gd_aliases&col=gd_pub_acc_ids&col=gd_locus_type&col=gd_date_mod&col=family.id&col=gd_locus_group&col=gd_name_aliases&col=gd_date_sym_change&col=gd_pub_eg_id&col=family.name&col=gd_date_name_change&col=gd_prev_name&col=gd_date2app_or_res&status=Approved&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit"
hgnc_data = []
r = requests.get(hgnc_data_url, stream=True)
lines = (line.decode("utf-8") for line in r.iter_lines())
for record in csv.DictReader(lines, delimiter="\t"):
    hgnc_data.append(record)

NameError: name 'requests' is not defined

TODO: has the `with requests.get...:` format stopped working? Did it never work?

In [ ]:
len(hgnc_data)

In [ ]:
hgnc_data[10]

In [ ]:
protein_hgnc_data = []
for record in hgnc_data:
    if record["Locus group"] == "protein-coding gene":
        protein_hgnc_data.append(record)
len(protein_hgnc_data)

What kinds of characters connect the base to the integer suffix?

In [ ]:
connector_re = re.compile("(.*?[a-zA-Z])(\W+?)?(\d+)$")
connectors = set()
for record in protein_hgnc_data:
    symbols = set([record["Approved symbol"]])
    for s in record["Previous symbols"].split(", "):
        symbols.add(s)
    for s in record["Alias symbols"].split(", "):
        symbols.add(s)

    for s in symbols:
        m = connector_re.match(s)
        if m:
            connector = m.group(2)
            int_suffix = int(m.group(3))
            connectors.add(connector)
            if not connector in [None, "-", "."]:
                print(s)
print(connectors)

In [ ]:
int_suffix_re = re.compile("(.*?[a-zA-Z][\-\/\.#\)\:]?)(\d+)$")
m = int_suffix_re.match("M-12219")

if m:
    print(m.group(1))
    print(m.group(2))

In [ ]:
gene_terms = set()
gene_term_bases = set()
int_suffixes_by_gene_term_base = dict()
for record in protein_hgnc_data:
    symbols = set([record["Approved symbol"]])
    for s in record["Previous symbols"].split(", "):
        symbols.add(s)
    for s in record["Alias symbols"].split(", "):
        symbols.add(s)

    for s in symbols:
        gene_terms.add(s)
        m = int_suffix_re.match(s)
        if m:
            gene_term_base = m.group(1)
            gene_term_bases.add(gene_term_base)
            int_suffix = int(m.group(2))
            if gene_term_base in int_suffixes_by_gene_term_base:
                int_suffixes_by_gene_term_base[gene_term_base].add(int_suffix)
            else:
                int_suffixes_by_gene_term_base[gene_term_base] = set(
                    [int_suffix]
                )
        else:
            gene_term_bases.add(s)
    approved_name = record["Approved name"]
    gene_term_bases.add(approved_name)
    gene_terms.add(approved_name)
    if "Alias names" in record and record["Alias names"]:
        for name in record["Alias names"].split(', "'):
            gene_terms.add(name)
            gene_term_bases.add(name.strip(' "'))
    if "Previous name" in record and record["Previous name"]:
        for name in record["Previous name"].split(', "'):
            gene_terms.add(name)
            gene_term_bases.add(name.strip(' "'))

In [ ]:
len(int_suffixes_by_gene_term_base.keys())

In [ ]:
len(gene_term_bases)

In [ ]:
len(gene_terms)

In [ ]:
ocr_data = list()
with open(
    "/home/ariutta/Documents/pfocr/data/ocr_pfocr20200224_raw_dollar_tail50.json"
) as f:
    for line in f:
        start = line[:2]
        end = line[-2:]
        if start == "{}":
            print("Detected a line representing an empty object {}")
            continue
        elif start != "${":
            print(line[:10])
            raise Exception(f"Line started with {start}, not a dollar sign.")
        elif end != "$\n":
            print(line[-10:])
            raise Exception(f"Line started with {end}, not a dollar sign.")
        ocr_data.append(json.loads(line.strip("$\n")))
print(ocr_data[0])

In [ ]:
print(ocr_data[1]["fullTextAnnotation"]["text"])

In [ ]:
ocr_data[5]["fullTextAnnotation"]["text"]

In [ ]:
def normalize_confusable(confusable_set, text):
    first_confusable = list(confusable_set)[0]
    normalized = text
    for confusable in list(confusable_set):
        if confusable in normalized:
            normalized = normalized.replace(confusable, first_confusable)
    return normalized

In [ ]:
gene_automaton = ahocorasick.Automaton()
confusable_sets = [set(["I", "l", "|", "1"])]
for idx, gene_term_base in enumerate(gene_term_bases):
    if len(gene_term_base) > 2:
        gene_term_base_cf = gene_term_base.casefold()
        gene_automaton.add_word(
            gene_term_base, (gene_term_base, "none", gene_term_base)
        )
        gene_automaton.add_word(
            gene_term_base_cf, (gene_term_base, "cf", gene_term_base_cf)
        )
        for confusable_set in confusable_sets:
            first_confusable = list(confusable_set)[0]
            normalized = normalize_confusable(confusable_set, gene_term_base)
            if normalized != gene_term_base:
                gene_automaton.add_word(
                    normalized,
                    (
                        gene_term_base,
                        f"{','.join(list(confusable_set))} -> {first_confusable}",
                        normalized,
                    ),
                )
                normalized_cf = normalized.casefold()
                if normalized_cf != normalized:
                    gene_automaton.add_word(
                        normalized_cf,
                        (
                            gene_term_base,
                            f"{','.join(list(confusable_set))} -> {first_confusable},cf",
                            normalized_cf,
                        ),
                    )
gene_automaton.make_automaton()

In [ ]:
numeric_suffix_re = re.compile(
    "^[\-\s]?(\d+)(\s?([\-,&\\/]|and|or)\s?(\d+))?([\s\.\|.*]|$)"
)
m = numeric_suffix_re.match("-5,6\nwow")
if m:
    # print(m.group(0))
    print(m.group(1))
    # print(m.group(2))
    print(m.group(3))
    print(m.group(4))

In [ ]:
test_re = re.compile("^.+?[\-\s]?(\d+)([A-Z]+)$")
for gene_term in list(gene_terms)[:100]:
    if test_re.match(gene_term):
        print(gene_term)

In [ ]:
int_associated_gene_terms = int_suffixes_by_gene_term_base.keys()
alpha_re = re.compile("[a-zA-Z]")
numeric_suffix_re = re.compile(
    "^[\-\s]?(\d+)(\s?([\-,&\\/]|and|or)\s?(\d+))?([\s\.\|.*]|$)"
)
# for data in [{"fullTextAnnotation": {"text": "LRP-5,6\nwow"}}]:
for data in ocr_data[:50]:
    matched = set()
    text = data["fullTextAnnotation"]["text"]
    print(f"-----{str(len(text))}------")
    for (
        end_index,
        (original_word, transforms, transformed_word),
    ) in gene_automaton.iter(text):
        matched.add(f"{str(end_index)},{original_word}")

        start_index = end_index - len(transformed_word) + 1
        if (alpha_re.match(text[start_index - 1])) or (
            alpha_re.match(text[end_index + 1])
        ):
            continue

        print("")
        if original_word in int_associated_gene_terms:
            print((end_index, (original_word, transforms, transformed_word)))
            print(
                (
                    text[max(0, start_index - 10) : start_index],
                    text[start_index : end_index + 1],
                    text[end_index + 1 : end_index + 20],
                )
            )
            start_index = end_index - len(transformed_word) + 1
            m = numeric_suffix_re.match(text[end_index + 1 : end_index + 20])
            if m:
                # print(m.group(0))
                # print(m.group(1))
                # print(m.group(2))
                # print(m.group(3))

                first_int = int(m.group(1))
                connector = m.group(3)
                last_int = m.group(4)
                if last_int:
                    last_int = int(last_int)

                print(first_int)
                print(connector)
                print(last_int)

                int_suffixes = int_suffixes_by_gene_term_base[original_word]
                # print(f"has associated ints: {int_suffixes}")

                if first_int in int_suffixes:
                    if (not connector is None) and last_int in int_suffixes:
                        print(
                            original_word
                            + str(first_int)
                            + connector
                            + str(last_int)
                        )
                        if connector in set(["-"]):
                            for i in range(first_int, last_int + 1):
                                all_matches = []
                                if i in int_suffixes:
                                    all_matches.append(original_word + str(i))
                                else:
                                    print(
                                        "This integer is not found in int_suffixes: "
                                        + str(i)
                                    )
                            print("-> " + ", ".join(all_matches))
                        else:
                            print(
                                "-> "
                                + original_word
                                + str(first_int)
                                + ", "
                                + original_word
                                + str(last_int)
                            )
                    else:
                        print(original_word + str(first_int))
                    continue
                else:
                    print(
                        "This integer is not found in int_suffixes: "
                        + str(first_int)
                    )
                    print(int_suffixes)
        if original_word in gene_terms:
            1 + 1
            # print("good on its own")
    text_cf = text.casefold()
    if text != text_cf:
        for (
            end_index,
            (original_word, transforms, transformed_word),
        ) in gene_automaton.iter(text_cf):
            start_index = end_index - len(transformed_word) + 1
            if (alpha_re.match(text[start_index - 1])) or (
                alpha_re.match(text[end_index + 1])
            ):
                continue

            match_str = f"{str(end_index)},{original_word}"
            if not match_str in matched:
                # print("* text_cf")
                matched.add(match_str)
                # print((end_index, (original_word, transforms, transformed_word)))

    for confusable_set in confusable_sets:
        normalized = normalize_confusable(confusable_set, text)
        if normalized != text:
            for (
                end_index,
                (original_word, transforms, transformed_word),
            ) in gene_automaton.iter(normalized):
                start_index = end_index - len(transformed_word) + 1
                if (alpha_re.match(text[start_index - 1])) or (
                    alpha_re.match(text[end_index + 1])
                ):
                    continue

                match_str = f"{str(end_index)},{original_word}"
                if not match_str in matched:
                    # print("* normalized")
                    matched.add(match_str)
                    # print((end_index, (original_word, transforms, transformed_word)))

            normalized_cf = normalized.casefold()
            if normalized_cf != normalized:
                for (
                    end_index,
                    (original_word, transforms, transformed_word),
                ) in gene_automaton.iter(normalized_cf):
                    start_index = end_index - len(transformed_word) + 1
                    if (alpha_re.match(text[start_index - 1])) or (
                        alpha_re.match(text[end_index + 1])
                    ):
                        continue

                    match_str = f"{str(end_index)},{original_word}"
                    if not match_str in matched:
                        # print("* normalized_cf")
                        matched.add(match_str)
                        # print((end_index, (original_word, transforms, transformed_word)))

In [ ]:
# gene_term_base_re = re.compile("(" + ")|(".join(["wt", "boo", "fus", "ss"]) + ")")

munged_gene_term_bases = set()
for gene_term_base in gene_term_bases:
    munged_gene_term_bases.add(gene_term_base.casefold())

escaped_munged_gene_term_bases = list()
for gene_term_base in sorted(munged_gene_term_bases, key=len, reverse=True):
    if len(gene_term_base) > 2:
        escaped_munged_gene_term_bases.append(re.escape(gene_term_base))

gene_term_base_re = re.compile(
    "(" + ")|(".join(list(escaped_munged_gene_term_bases)) + ")"
)

In [ ]:
escaped_munged_gene_term_bases[-1]

In [ ]:
for m in re.finditer(
    gene_term_base_re, ocr_data[0]["fullTextAnnotation"]["text"].casefold()
):
    if m.group(0):
        print(m.start(), m.end(), m.group(0))
        # print("")

The cells below were previously trying to use `re2`, not `re`. I think this was an experiment with using a fast regular expression library. But if I'm using the Aho-Corasick algo, I may not need to use the code below at all.

In [ ]:
gene_term_base_re = re.compile(
    "(" + ")|(".join(list(escaped_munged_gene_term_bases[:5])) + ")"
)
len(escaped_munged_gene_term_bases)

In [ ]:
for i in range(0, 74231, 10000):
    print(i)

In [ ]:
prev = 0
for i in range(0, 90000, 10000):
    for m in re.finditer(
        "(" + ")|(".join(list(escaped_munged_gene_term_bases[prev:i])) + ")",
        ocr_data[0]["fullTextAnnotation"]["text"].casefold(),
    ):
        if m.group(0):
            print(m.start(), m.end(), m.group(0))
            # print("")
    prev = i
print("hello")

In [ ]:
gene_term_base_res = []
prev = 0
for i in range(0, 90000, 10000):
    gene_term_base_re = re.compile(
        "(" + ")|(".join(list(escaped_munged_gene_term_bases[prev:i])) + ")"
    )
    gene_term_base_res.append(gene_term_base_re)
    prev = i

for gene_term_base_re in gene_term_base_res:
    for data in ocr_data[2:4]:
        text = data["fullTextAnnotation"]["text"]
        # print(text)
        print(f"-----{str(len(text))}------")
        for m in gene_term_base_re.finditer(text.casefold()):
            if m.group(0):
                print(m.start(), m.end(), m.group(0))
                # print("")
print("hello")

In [ ]:
gene_term_base_res = []
prev = 0
for i in range(0, 90000, 10000):
    gene_term_base_re = re.compile(
        "(" + ")|(".join(list(escaped_munged_gene_term_bases[prev:i])) + ")"
    )
    gene_term_base_res.append(gene_term_base_re)
    prev = i
for data in ocr_data[2:4]:
    text = data["fullTextAnnotation"]["text"]
    # print(text)
    print(f"-----{str(len(text))}------")
    for gene_term_base_re in gene_term_base_res:
        for m in gene_term_base_re.finditer(text.casefold()):
            if m.group(0):
                print(m.start(), m.end(), m.group(0))
                # print("")
print("hello")

In [ ]:
gene_term_base_re = re.compile(
    "(" + ")|(".join(list(escaped_munged_gene_term_bases)) + ")"
)
for data in ocr_data[2:4]:
    text = data["fullTextAnnotation"]["text"]
    # print(text)
    print("")
    for m in gene_term_base_re.finditer(text.casefold()):
        if m.group(0):
            print(m.start(), m.end(), m.group(0))
            # print("")

In [ ]:
data.keys()

In [ ]:
data["fullTextAnnotation"]["text"]

In [ ]:
for item in data["fullTextAnnotation"]["pages"]:
    print(item.keys())

In [ ]:
for item in data["fullTextAnnotation"]["pages"][0]["blocks"]:
    print(item.keys())

In [ ]:
for item in data["fullTextAnnotation"]["pages"][0]["blocks"][0]["paragraphs"]:
    print(item.keys())

In [ ]:
for item in data["fullTextAnnotation"]["pages"][0]["blocks"][0]["paragraphs"][
    0
]["words"]:
    print(item.keys())

In [ ]:
for page in data["fullTextAnnotation"]["pages"]:
    for block in page["blocks"]:
        for paragraph in block["paragraphs"]:
            for word in paragraph["words"]:
                for symbol in word["symbols"]:
                    print(symbol.keys())
                    print(symbol["text"])

In [ ]:
for boo in data["textAnnotations"]:
    print(boo.keys())

In [ ]:
data["textAnnotations"][0]["description"]

In [ ]:
data["textAnnotations"][1]

In [ ]:
data["textAnnotations"][1]["description"]